Code Cells realisées par BAHASSOU Imane(N°2):
    [1],[2],[3]
Code Cells realisées par BOUTALLOUST Mohammed Amine(N°6):
    [4],[5],[6]

In [1]:
from typing import Dict, List, Tuple, Optional

In [2]:
# Task information: task_id -> (duration, [predecessors], description)
tasks = {
    "T1": (30, [], "Chassis Assembly"),
    "T2": (40, ["T1"], "Engine Installation"),
    "T3": (60, ["T1"], "Painting"),
    "T4": (50, ["T2"], "Electrical Wiring"),
    "T5": (45, ["T3"], "Interior Installation"),
    "T6": (30, ["T4", "T5"], "Quality Control"),
}

MAX_PARALLEL_TASKS = 2

# A schedule maps task_id -> (start_time, end_time)
Schedule = Dict[str, Tuple[int, int]]

In [3]:
def is_consistent(task: str, start_time: int, assignment: Schedule) -> bool:
    duration = tasks[task][0]
    end_time = start_time + duration

    # Worker/resource constraint
    overlapping = 0
    for _, (s, e) in assignment.items():
        if start_time < e and end_time > s:
            overlapping += 1
        if overlapping >= MAX_PARALLEL_TASKS:
            return False

    # Precedence constraint
    for pred in tasks[task][1]:
        if pred not in assignment:
            return False
        if start_time < assignment[pred][1]:
            return False

    return True

In [4]:
def select_unassigned_variable(assignment: Schedule) -> Optional[str]:
    for task in tasks:
        if task not in assignment:
            return task
    return None

def order_domain_values(task: str, assignment: Schedule) -> List[int]:
    # Try starting every 5 minutes up to 8 hours
    return list(range(0, 8 * 60, 5))

def inference(task: str, time: int, assignment: Schedule) -> Optional[Dict[str, Tuple[int, int]]]:
    # No inference in this basic implementation
    return {}

In [5]:
def backtrack(assignment: Schedule) -> Optional[Schedule]:
    if len(assignment) == len(tasks):
        return assignment

    var = select_unassigned_variable(assignment)
    if var is None:
        return None

    for value in order_domain_values(var, assignment):
        if is_consistent(var, value, assignment):
            assignment[var] = (value, value + tasks[var][0])
            inferences = inference(var, value, assignment)

            if inferences is not None:
                # Merge inferences
                assignment.update(inferences)
                result = backtrack(assignment)
                if result:
                    return result
                # Remove inferences
                for inferred in inferences:
                    assignment.pop(inferred)
            assignment.pop(var)

    return None

def backtracking_search() -> Optional[Schedule]:
    return backtrack({})


In [6]:
def main():
    solution = backtracking_search()
    if solution:
        for task, (start, end) in sorted(solution.items(), key=lambda x: x[1][0]):
            description = tasks[task][2]
            print(f"{task} ({description}): {start} -> {end} minutes")
    else:
        print("No valid schedule found.")

if __name__ == "__main__":
    main()

T1 (Chassis Assembly): 0 -> 30 minutes
T2 (Engine Installation): 30 -> 70 minutes
T3 (Painting): 30 -> 90 minutes
T4 (Electrical Wiring): 70 -> 120 minutes
T5 (Interior Installation): 90 -> 135 minutes
T6 (Quality Control): 135 -> 165 minutes
